In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score,classification_report
from sklearn.svm import SVC

In [2]:
salary_test = pd.read_csv("/content/SalaryData_Test(1).csv")
salary_test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [3]:
salary_train = pd.read_csv("/content/SalaryData_Train(1).csv")
salary_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
salary_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [5]:
salary_train.isna().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [6]:
salary_train[salary_train.duplicated()]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
326,33,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
364,27,Private,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,50,United-States,<=50K
518,24,Private,HS-grad,9,Never-married,Handlers-cleaners,Unmarried,Black,Female,0,0,40,United-States,<=50K
525,24,Private,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
619,33,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30110,47,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
30118,20,Private,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States,<=50K
30132,22,Private,Some-college,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K


observed duplicated records lets do build base model without handling duplicates

In [7]:
salary_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [8]:
salary_test[salary_test.duplicated()]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
551,59,Private,HS-grad,9,Divorced,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K
601,32,Private,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K
681,40,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
799,24,Private,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,35,United-States,<=50K
1044,44,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,60,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15043,51,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
15044,22,Private,Some-college,10,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
15049,32,Private,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States,<=50K
15052,31,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K


In [9]:
# target variabe is salary
salary_train['Salary'].value_counts()

 <=50K    22653
 >50K      7508
Name: Salary, dtype: int64

In [10]:
# converting categorical columns into numerical using label encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
# train data
for feature in salary_train.columns:
  if salary_train[feature].dtype == 'object':
    salary_train[feature] = encoder.fit_transform(salary_train[feature])
# test data
for feature in salary_test.columns:
  if salary_test[feature].dtype == 'object':
    salary_test[feature] = encoder.fit_transform(salary_test[feature])

In [11]:
salary_train['Salary'].value_counts()

0    22653
1     7508
Name: Salary, dtype: int64

In [12]:
# separting target variable from data(train and test)
X_train = salary_train.iloc[:,:-1]
y_train = salary_train.iloc[:,-1]
X_test = salary_test.iloc[:,:-1]
y_test = salary_test.iloc[:,-1]

# model building using SVM

In [13]:
BaseModel = SVC()
BaseModel.fit(X_train,y_train)

SVC()

In [14]:
print("train score",BaseModel.score(X_train,y_train))
print("test score",BaseModel.score(X_test,y_test))

train score 0.7960279831570571
test score 0.7964143426294821


In [15]:
print(classification_report(y_test,BaseModel.predict(X_test)))

              precision    recall  f1-score   support

           0       0.80      0.97      0.88     11360
           1       0.73      0.27      0.39      3700

    accuracy                           0.80     15060
   macro avg       0.77      0.62      0.64     15060
weighted avg       0.79      0.80      0.76     15060



#Base model recall and f1 score of class 1 is low.

In [16]:
#lets drop duplicated records and  see metrics
salary_train = salary_train.drop_duplicates()
salary_train.reset_index(drop=True)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,5,9,13,4,0,1,4,1,2174,0,40,37,0
1,50,4,9,13,2,3,0,4,1,0,0,13,37,0
2,38,2,11,9,0,5,1,4,1,0,0,40,37,0
3,53,2,1,7,2,5,0,2,1,0,0,40,37,0
4,28,2,9,13,2,9,5,2,0,0,0,40,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26898,53,2,12,14,2,3,0,4,1,0,0,40,37,1
26899,22,2,15,10,4,10,1,4,1,0,0,40,37,0
26900,27,2,7,12,2,12,5,4,0,0,0,38,37,0
26901,58,2,11,9,6,0,4,4,0,0,0,40,37,0


In [17]:
salary_test =  salary_test.drop_duplicates()
salary_test.reset_index(drop=True)

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,2,1,7,4,6,3,2,1,0,0,40,37,0
1,38,2,11,9,2,4,0,4,1,0,0,50,37,0
2,28,1,7,12,2,10,0,4,1,0,0,40,37,1
3,44,2,15,10,2,6,0,2,1,7688,0,40,37,1
4,34,2,0,6,4,7,1,4,1,0,0,30,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14125,33,2,9,13,4,9,3,4,1,0,0,40,37,0
14126,39,2,9,13,0,9,1,4,0,0,0,36,37,0
14127,38,2,9,13,2,9,0,4,1,0,0,50,37,0
14128,44,2,9,13,0,0,3,1,1,5455,0,40,37,0


In [18]:
# separting target variable from data(train and test)
X_train = salary_train.iloc[:,:-1]
y_train = salary_train.iloc[:,-1]
X_test = salary_test.iloc[:,:-1]
y_test = salary_test.iloc[:,-1]

In [19]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [20]:
X_train = norm_func(X_train)
X_test =  norm_func(X_test)
y_train = y_train
y_test = y_test

In [21]:
# Base model without duplicat records
BaseModel2 = SVC()
BaseModel2.fit(X_train,y_train)
print("train score", BaseModel2.score(X_train,y_train))
print("test score", BaseModel2.score(X_test,y_test))

train score 0.8399806713005984
test score 0.8410474168435952


In [22]:
print(classification_report(y_test,BaseModel2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90     10620
           1       0.75      0.54      0.63      3510

    accuracy                           0.84     14130
   macro avg       0.81      0.74      0.76     14130
weighted avg       0.83      0.84      0.83     14130



handling duplicated data and normalizing increased model accuracy, recall and F1 score of class 1.

In [23]:
model_linear = SVC(kernel = "linear")
model_linear.fit(X_train,y_train)

SVC(kernel='linear')

In [24]:
print(classification_report(y_test,model_linear.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.97      0.88     10620
           1       0.77      0.30      0.44      3510

    accuracy                           0.80     14130
   macro avg       0.79      0.64      0.66     14130
weighted avg       0.80      0.80      0.77     14130



In [25]:
model_rbf = SVC(kernel = "rbf")
model_rbf.fit(X_train,y_train)
print(classification_report(y_test,model_rbf.predict(X_test)))


              precision    recall  f1-score   support

           0       0.86      0.94      0.90     10620
           1       0.75      0.54      0.63      3510

    accuracy                           0.84     14130
   macro avg       0.81      0.74      0.76     14130
weighted avg       0.83      0.84      0.83     14130



In [26]:
model_poly = SVC(kernel = "poly")
model_poly.fit(X_train,y_train)

print(classification_report(y_test,model_poly.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90     10620
           1       0.75      0.55      0.63      3510

    accuracy                           0.84     14130
   macro avg       0.81      0.74      0.77     14130
weighted avg       0.83      0.84      0.83     14130



In [ ]:
model_sigmoid = SVC(kernel = "sigmoid")
model_sigmoid.fit(X_train,y_train)


In [28]:
print(classification_report(y_test,model_sigmoid.predict(X_test)))

              precision    recall  f1-score   support

           0       0.72      0.72      0.72     10620
           1       0.14      0.14      0.14      3510

    accuracy                           0.57     14130
   macro avg       0.43      0.43      0.43     14130
weighted avg       0.57      0.57      0.57     14130



#conclusion: comparing all model recall and accuracy model_poly giving high scores.